In [0]:
import time
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
# from plotly import tools
# from plotly.graph_objs import *
# from plotly.offline import init_notebook_mode, iplot, iplot_mpl
from google.colab import drive
import random
import matplotlib.pyplot as plt
import seaborn as sns

#random.seed(30)
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
#threshold = input()
!ls '/content/drive/My Drive/'

 1.png
 2.png
 3.png
 4.png
 BIA_660_final_report_24_4.ipynb
 Bonus1
 Bonus2
 Chase_mortgage_new.gsheet
'Colab Notebooks'
 DL_final_project
'DRAFT LIST of STUDENT POSTERS April 28 2019.gsheet'
'DRAFT LIST of STUDENT POSTERS April 28 2019.xlsx'
 fake_news_only_body.csv
 fake_news_only_statement.csv
'FE541 Project Proposal Group 6.docx'
'FE541 Project Proposal Group 6.gdoc'
'Getting started.pdf'
 HW3
 hw4_and_bonus
 IpullRank
 merged_data.csv
 ML_proj
 politifact_fake_statements.csv
 Project_Presentation_description.gdoc
 real_news_only_body.csv
 real_news_only_statements.csv
'Ronald_Fernandes_Resume (1).pdf'
'Ronald_Fernandes_Resume (2).pdf'
 Ronald_Fernandes_Resume.pdf
'Ronald_Fernandes_Resume - Ronald Fernandes.pdf'
'R test'
 Shalini_FE800
 test
 WebMining_Lecture_notes.zip
 X.npy
 Y.npy


In [0]:
data = pd.read_csv('/content/drive/My Drive/merged_data.csv')
data = data.set_index('Date')
data.head()

,deb,exp,gdp,hou,iip,inf,int,oil,s&p,une,dxy,historical
Date,,,,,,,,,,,,
1/1/2006,8170.42,78746.0,2.6,0.6,0.62,3.4,4.25,61.04,1248.29,4.9,91.080002,16873.85235
1/2/2006,8170.42,78746.0,2.6,0.6,0.62,3.4,4.25,61.04,1248.29,4.9,91.080002,16873.85235
1/3/2006,8153.88,78746.0,2.6,0.6,0.62,3.4,4.25,63.14,1268.80,4.9,90.750000,17151.09779
1/4/2006,8165.86,78746.0,2.6,0.6,0.62,3.4,4.25,63.42,1273.46,4.9,89.730003,17214.08968
1/5/2006,8160.32,78746.0,2.6,0.6,0.62,3.4,4.25,62.79,1273.48,4.9,89.269997,17214.36003


In [0]:
data.describe()

,deb,exp,gdp,hou,iip,inf,int,oil,s&p,une,dxy,historical
count,5113.000000,5113.000000,5113.000000,5113.000000,5113.000000,5113.000000,5113.000000,5113.000000,5113.000000,5113.000000,5113.000000,5113.000000
mean,15542.373278,118336.879261,1.789341,0.156679,0.058115,1.961275,1.403872,73.178723,1755.394324,6.185625,86.262518,23728.672534
std,4480.572454,19264.330864,2.309371,0.535591,0.700897,1.336611,1.664744,22.211364,607.314000,2.028167,8.222797,8209.411884
min,8153.880000,74208.000000,-8.400000,-1.800000,-4.340000,-2.100000,0.250000,26.210000,676.530000,3.500000,71.330002,9145.044283
25%,11490.990000,104826.000000,0.900000,-0.100000,-0.270000,1.200000,0.250000,54.500000,1288.220000,4.600000,79.889999,17413.609070
50%,16431.220000,124713.000000,2.200000,0.300000,0.120000,2.000000,0.500000,70.460000,1525.200000,5.400000,83.959999,20617.003740
75%,19316.930000,133491.000000,3.200000,0.500000,0.440000,2.700000,2.000000,92.690000,2124.290000,8.200000,94.849998,28715.247100
max,23201.380000,150429.000000,5.500000,1.000000,1.520000,5.600000,5.250000,145.290000,3240.020000,10.000000,103.260002,43797.209850


In [0]:
# data = pd.read_csv('./Data/Stocks/goog.us.txt')
# data['Date'] = pd.to_datetime(data['Date'])
# data = data.set_index('Date')
# print(data.index.min(), data.index.max())
# data.head()
#data.iloc[1]['historical']

In [0]:
date_split = '1/1/2018'
train = data[:date_split]
test = data[date_split:]
len(train),len(test)

(4384, 730)

In [0]:
# def plot_train_test(train, test, date_split):
    
#     data = [
#         Candlestick(x=train.index, open=train['Open'], high=train['High'], low=train['Low'], close=train['Close'], name='train'),
#         Candlestick(x=test.index, open=test['Open'], high=test['High'], low=test['Low'], close=test['Close'], name='test')
#     ]
#     layout = {
#          'shapes': [
#              {'x0': date_split, 'x1': date_split, 'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper', 'line': {'color': 'rgb(0,0,0)', 'width': 1}}
#          ],
#         'annotations': [
#             {'x': date_split, 'y': 1.0, 'xref': 'x', 'yref': 'paper', 'showarrow': False, 'xanchor': 'left', 'text': ' test data'},
#             {'x': date_split, 'y': 1.0, 'xref': 'x', 'yref': 'paper', 'showarrow': False, 'xanchor': 'right', 'text': 'train data '}
#         ]
#     }
#     figure = Figure(data=data, layout=layout)
#     iplot(figure)

In [0]:
#plot_train_test(train, test, date_split)
#[[0 for _ in range(10)] for i in range(90)]

#[[1,2]] + [[3,4],[5,6]]

In [0]:
class Environment1:
    
    def __init__(self, data, history_t=90):
        self.data = data
        self.history_t = history_t
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        self.profits = 0
        self.positions = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs
    
    def step(self, act):
        reward = 0
        
        # act = 0: stay, 1: buy, 2: sell
        if act == 1: # buy
            self.positions.append(self.data.iloc[self.t, :]['s&p'])
        elif act == 2: # sell
            if len(self.positions) == 0:
                reward = -1
            else:
                profits = 0
                for p in self.positions:
                    profits += (self.data.iloc[self.t, :]['s&p'] - p)
                reward += profits
                self.profits += profits
                self.positions = []
        
        # set next time
        self.t += 1
        
        self.position_value = 0
        for p in self.positions:
            self.position_value += (self.data.iloc[self.t, :]['s&p'] - p)
        self.history.pop(0)
        self.history.append(self.data.iloc[self.t, :]['s&p'] - self.data.iloc[(self.t-1), :]['s&p'])
        if (self.t==len(self.data)-1):
            self.done=True
        # clipping reward
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        #print ("t={%d}, done={%str}"%(self.t,self.done))
        return [self.position_value] + self.history, reward, self.done # obs, reward, done
    

In [0]:
env = Environment1(train)
print(env.reset())
for _ in range(3):
    pact = np.random.randint(3)
    print (pact)
    print(env.step(pact))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0], -1, False)
2
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 20.50999999999999], -1, False)
0
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [0]:
#def train_dqn(env):
    
class Q_Network(nn.Module):
        
  def __init__(self,obs_len,hidden_size,actions_n):
            
        super(Q_Network,self).__init__()
            
        self.fc_val = nn.Sequential(
            nn.Linear(obs_len, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, actions_n)
        )
        
        
  def forward(self,x):
        h =  self.fc_val(x)
        return (h)
            

In [0]:
hidden_size=100
input_size=env.history_t+1
output_size=3
USE_CUDA = False 
LR = 0.001

In [0]:
Q = Q_Network(input_size, hidden_size, output_size)

In [0]:
Q_ast = copy.deepcopy(Q)

In [0]:
if USE_CUDA:
    Q = Q.cuda()
loss_function = nn.MSELoss()
optimizer = optim.Adam(list(Q.parameters()), lr=LR)

In [0]:
epoch_num = 20
step_max = len(env.data)-1
memory_size = 200
batch_size = 50
gamma = 0.97

In [0]:
obs, reward, done = env.step(5)

In [0]:
memory = []
total_step = 0
total_rewards = []
total_losses = []
epsilon = 1.0
epsilon_decrease = 1e-3
epsilon_min = 0.1
start_reduce_epsilon = 200
train_freq = 10
update_q_freq = 20
gamma = 0.97
show_log_freq = 5

In [0]:
#Q(torch.from_numpy(np.array(pobs, dtype=np.float32)))

In [0]:
thresholds_list = [5000,10000,20000,25000,40000,45000]
profits_list = []
for thresh in thresholds_list:
  start = time.time()
  threshold = thresh
  for epoch in range(epoch_num):

      pobs = env.reset()
      step = 0
      done = False
      total_reward = 0
      total_loss = 0
      count = 0
      i = 0
      while not done and step < step_max:

          # select act
          
          pact = np.random.randint(3)
          if np.random.rand() > epsilon:
              pact = Q(torch.from_numpy(np.array(pobs, dtype=np.float32)))
              #print(pact)
              if(data.iloc[i]['historical'] <= threshold):
                pact = np.argmax(pact.data)
                #print(pact)
                pact = pact.numpy()
                #print(pact)
              else:
                pact = 0
              # count = count + 1
              # print(count)


          # act
          obs, reward, done = env.step(pact)

          # add memory
          memory.append((pobs, pact, reward, obs, done))
          if len(memory) > memory_size:
              memory.pop(0)

          # train or update q
          if len(memory) == memory_size:
              if total_step % train_freq == 0:
                  shuffled_memory = np.random.permutation(memory)
                  memory_idx = range(len(shuffled_memory))
                  for i in memory_idx[::batch_size]:
                      batch = np.array(shuffled_memory[i:i+batch_size])
                      b_pobs = np.array(batch[:, 0].tolist(), dtype=np.float32).reshape(batch_size, -1)
                      b_pact = np.array(batch[:, 1].tolist(), dtype=np.int32)
                      b_reward = np.array(batch[:, 2].tolist(), dtype=np.int32)
                      b_obs = np.array(batch[:, 3].tolist(), dtype=np.float32).reshape(batch_size, -1)
                      b_done = np.array(batch[:, 4].tolist(), dtype=np.bool)

                      q = Q(torch.from_numpy(b_pobs))
                      q_ = Q_ast(torch.from_numpy(b_obs))
                      maxq = np.max(q_.data.numpy(),axis=1)
                      target = copy.deepcopy(q.data)
                      for j in range(batch_size):
                          target[j, b_pact[j]] = b_reward[j]+gamma*maxq[j]*(not b_done[j])
                      Q.zero_grad()
                      loss = loss_function(q, target)
                      total_loss += loss.data.item()
                      loss.backward()
                      optimizer.step()
                      
              if total_step % update_q_freq == 0:
                  Q_ast = copy.deepcopy(Q)
                  
              # epsilon
              if epsilon > epsilon_min and total_step > start_reduce_epsilon:
                  epsilon -= epsilon_decrease

              # next step
              total_reward += reward
              pobs = obs
              step += 1
              total_step += 1

          total_rewards.append(total_reward)
          total_losses.append(total_loss)

          if (epoch+1) % show_log_freq == 0:
              log_reward = sum(total_rewards[((epoch+1)-show_log_freq):])/show_log_freq
              log_loss = sum(total_losses[((epoch+1)-show_log_freq):])/show_log_freq
              elapsed_time = time.time()-start
              #print('\t'.join(map(str, [epoch+1, epsilon, total_step, log_reward, log_loss, elapsed_time])))
              start = time.time()
          i = i + 1    
  #return Q, total_losses, total_rewards
  test_env = Environment1(test)
  pobs = test_env.reset()
  test_acts = []
  test_rewards = []

  for _ in range(len(test_env.data)-1):
      
      pact = Q(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
      pact = np.argmax(pact.data)
      test_acts.append(pact.item())
              
      obs, reward, done = test_env.step(pact.numpy())
      test_rewards.append(reward)

      pobs = obs
          
  test_profits = test_env.profits
  profits_list.append(test_profits)

In [0]:
# test_env = Environment1(test)
# pobs = test_env.reset()
# test_acts = []
# test_rewards = []

# for _ in range(len(test_env.data)-1):
    
#     pact = Q(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
#     pact = np.argmax(pact.data)
#     test_acts.append(pact.item())
            
#     obs, reward, done = test_env.step(pact.numpy())
#     test_rewards.append(reward)

#     pobs = obs
        
# test_profits = test_env.profits

In [0]:
#test_profits
print(thresholds_list)
print(profits_list)
#thresholds_list

In [0]:
tl = [5000,10000,20000,25000,40000,45000]
pl = [1431.41,-1699.26,465.82,981.75,105.74,839.04]
plt.figure(figsize=(10,5))
#plt.xticks(np.arange(6),tl)
sns.barplot(tl,pl,palette=['Green','Red','Green','Green','Green','Green'])
plt.xlabel('Threshold')
plt.ylabel('Profit')
plt.show()



In [0]:
print(pd.DataFrame({'Threshold':thresholds_list,'Profits':profits_list}))

In [0]:
# class Environment:
#   def __init__(self,data,history_t = 90):
#     #Initializes the class variable
  
#   def reset(self):
#     #Resets the environment
  
#   def step(self,act)
#     #For every step checks what action was taken
#     #Calculates Profits based on the actions
#     #Calculates rewards based on the profits

# class Q_Network(nn.Module):   
#   def __init__(self,obs_len,hidden_size,actions_n):
#         #Initializes the class variables
#         super(Q_Network,self).__init__()
#         #Initializes the neural Network Module 

# #Training
# for every Epoch:
#   #Train and get action for every row
#   #Calculate the reward for every step
#   #Calculate maximum reward for all possible states of next action
#   #Update model of q network for passing the values

# #Testing
#   #Test for every value what action should be taken
#   #Calculate total profits
